In [1]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import openpyxl as yxl
import pandas as pd
import os
import numpy as np

In [2]:
#Reference: https://strategyanalytics.medium.com/pandas-read-excel-removed-support-for-xlsx-files-426e4acfde89
df = pd.read_excel('CleanCountyProfile.xlsx', engine='openpyxl')


In [3]:
#O=increasing, 1=decreasing
data = df.reset_index(drop=True)
target1 = data["Land Class"]
target2 = data["GHG Rate"]
target_names = ["0", "1"]
data = data.drop(["GHG Rate", "Land Class", "County", "County.1", "Rural / Urban Status"], axis=1)
data

,Count of Democratic Voters,Count of Republican Voters,Count of No Affiliation Voters,"Population, 1900","Population, 1950","Population, 1990","Population, 2000","Population, 2010","% Change, 2000-10","Population, 2019",...,"Farm Sales Under $10,000, 2017","Farm Sales $10,000-$49,999, 2017","Farm Sales $50,000+, 2017","Preserved Farms, April 2017","Acres of Preserved Farmland, April 2017",Population,Birth,Cancer,Death Rate,GDP
0,19264,39814,7990,34496,44197,78274,91292,101407,0.110798,103009,...,0.458988,0.247818,0.293194,163,21434.02550,102811,17.9,460.2,743.3,3507548
1,528445,260457,91105,775058,1515237,1336449,1281666,1223348,-0.045502,1216045,...,0.681234,0.197943,0.120823,33,3458.34250,1218452,20.4,458.6,774.0,91296413
2,12585,25327,3157,52551,80842,73478,72392,68941,-0.047671,64735,...,0.567365,0.272455,0.160180,5,486.04900,65263,17.1,427.7,798.8,2322321
3,51464,46144,9440,56432,175192,186093,181412,170539,-0.059935,163929,...,0.678630,0.185971,0.135400,27,2932.56800,164742,18.7,467.5,836.0,7076904
4,6548,23043,2003,39468,40775,47919,49984,49762,-0.004441,47888,...,0.477998,0.235548,0.286454,17,3785.58625,48176,19.4,434.6,733.7,1448401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7794,14566,2317,38946,42698,45050,43863,41815,-0.046691,39191,...,0.634956,0.205752,0.159292,2,309.83900,39498,21.1,437.4,767.7,1766395
60,59728,64826,13138,92181,209628,204584,202897,207820,0.024264,206865,...,0.639773,0.273864,0.086364,34,5393.52390,207346,19.1,477.1,817.5,14054804
61,98169,117886,19054,160175,313179,370321,369993,365169,-0.013038,348899,...,0.586897,0.256597,0.156506,95,12616.65700,350611,16.3,447.1,798.1,13532650
62,5050,10537,1295,17152,16766,28076,28080,28276,0.006980,26794,...,0.612195,0.229268,0.158537,10,1569.47000,27046,17.8,429.5,954.6,2207240


In [4]:
#Dropping historical data, taking only information that is indicative of a more recent period of time for the state (2015-2020)
data = data.drop(["Population, 1900", "Population, 1950", "Population, 1900", "Population, 2000", "Population, 2010", "% Change, 2000-10"], axis=1)
data = data.drop(["Population, 1990", "Estimated # Births, 2010 to 2019", "Estimated # Deaths, 2010 to 2019"], axis=1)
data = data.drop(["# Rural Municipalities, 2010", "# Urban Municipalities, 2010" ,"Median Age, 1990", "Median Age, 2000",
                 "Median Age, 2010", "% Households with  One or More People <18 years old, 2010", "Average # Persons in Household, 1990",
                 "Average # Persons in Household, 2000", "Average # Persons in Household, 2010", "New Housing Units Construction Permits, 2017",
                 "New Housing Units Construction Permits, 2018", "Median Household Income, 1998", "Median Household Income, 2008"], axis=1)


In [5]:
data = data.drop(["Per Capita Personal Income, 1998", "Per Capita Personal Income, 2008", "Gross Domestic Product (in $ Millions), 2010", 
                 "Gross Domestic Product (in $ Millions), 2015", "Total Poverty Rate, 1998", "Total Poverty Rate, 2008", "Poverty Rate for Children (<18 years old), 1998",
                 "Poverty Rate for Children (<18 years old), 2008", "Population Receiving Cash Assistance, June 2008", "Population Receiving Cash Assistance, June 2015", 
                  "Population Eligible for Medical Assistance (MA), June 2008", "Population Eligible for Medical Assistance (MA), June 2015", 
                  "Population Participating in SNAP, June 2008", "Population Participating in SNAP, June 2015", "% Change in Expenditures Per Student, 2014 to 2016",
                 "# to Receive Free & Reduced Lunch, 2016", "# to Receive Free & Reduced Lunch, 2017", "Graduation Rate (4-year Cohort), 2015-16"], axis=1)

In [6]:
data = data.drop(["Graduation Rate (4-year Cohort), 2016-17", "Average Unemployment Rate, 2017", 
                 "Average Unemployment Rate, 2018", "Average # Business Establishments, 2nd Qt. 2017", "Average # Business Establishments, 2nd Qt. 2018", 
                 "Average Employment, 2nd Qt. 2017", "Average Employment, 2nd Qt. 2018", "Avg. Time to Work (Minutes), 1990", "Avg. Time to Work (Minutes), 2000", 
                 "Avg. Time to Work (Minutes), 2010", "Total # Filings for Chapters 7, 11, 12, and 13, 2017", "Total # Filings for Chapters 7, 11, 12, and 13, 2018", 
                 " Live Births Per 1,000 Residents, 1988", " Live Births Per 1,000 Residents, 1998", " Live Births Per 1,000 Residents, 2008",
                 " Deaths Per 1,000 Residents, 1988", " Deaths Per 1,000 Residents, 1998", " Deaths Per 1,000 Residents, 2008", 
                 "# Infant Deaths per 1,000 Live Births, 2000 to 2004", "%  Without Health Insurance <65 Years Old, 2007",
                 "%  Without Health Insurance <65 Years Old, 2013", "Serious Crimes per 100,000 Residents, 2007", 
                 "Serious Crimes per 100,000 Residents, 2013", "Full-Time Police Officers per 1,000 Residents, 2007",
                 "Full-Time Police Officers per 1,000 Residents, 2013", "Voter Turnout, Nov. 2004", "Voter Turnout, Nov. 2008", 
                 "Voter Turnout, Nov. 2012", "# Farms, 2007", "# Farms, 2012", "Acres in Farmland, 2007", "Acres in Farmland, 2012",
                 "% Land In Farms, 2007", "% Land In Farms, 2012", "Average Size of Farms (acres), 2007", "Average Size of Farms (acres), 2012"], axis=1)

In [7]:
feature_names = data.columns
data

,Count of Democratic Voters,Count of Republican Voters,Count of No Affiliation Voters,"Population, 2019","% Change, 2010-19","Natural Population Change (Births Minus Deaths), 2010 to 2019",Net International Migration,Net Domestic Migration,Total Net Migration,"Land Area in Square Miles, 2010",...,"Farm Sales Under $10,000, 2017","Farm Sales $10,000-$49,999, 2017","Farm Sales $50,000+, 2017","Preserved Farms, April 2017","Acres of Preserved Farmland, April 2017",Population,Birth,Cancer,Death Rate,GDP
0,19264,39814,7990,103009,0.015798,189,544,926,1470,518.667519,...,0.458988,0.247818,0.293194,163,21434.02550,102811,17.9,460.2,743.3,3507548
1,528445,260457,91105,1216045,-0.005970,-6146,27900,-28057,-157,730.074498,...,0.681234,0.197943,0.120823,33,3458.34250,1218452,20.4,458.6,774.0,91296413
2,12585,25327,3157,64735,-0.061009,-1807,15,-2492,-2477,653.202617,...,0.567365,0.272455,0.160180,5,486.04900,65263,17.1,427.7,798.8,2322321
3,51464,46144,9440,163929,-0.038759,-4294,161,-2292,-2131,434.712442,...,0.678630,0.185971,0.135400,27,2932.56800,164742,18.7,467.5,836.0,7076904
4,6548,23043,2003,47888,-0.037659,-844,51,-1058,-1007,1012.296348,...,0.477998,0.235548,0.286454,17,3785.58625,48176,19.4,434.6,733.7,1448401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7794,14566,2317,39191,-0.062753,-982,87,-1705,-1618,884.134793,...,0.634956,0.205752,0.159292,2,309.83900,39498,21.1,437.4,767.7,1766395
60,59728,64826,13138,206865,-0.004595,-5086,805,3532,4337,856.988913,...,0.639773,0.273864,0.086364,34,5393.52390,207346,19.1,477.1,817.5,14054804
61,98169,117886,19054,348899,-0.044555,-12702,626,-3780,-3154,1027.554384,...,0.586897,0.256597,0.156506,95,12616.65700,350611,16.3,447.1,798.1,13532650
62,5050,10537,1295,26794,-0.052412,-221,62,-1322,-1260,397.323492,...,0.612195,0.229268,0.158537,10,1569.47000,27046,17.8,429.5,954.6,2207240


In [8]:
for feature in feature_names: 
    print(feature)

Count of Democratic Voters
Count of Republican Voters
Count of No Affiliation Voters
Population, 2019
% Change, 2010-19
Natural Population Change (Births Minus Deaths), 2010 to 2019
Net International Migration
Net Domestic Migration
Total Net Migration
Land Area in Square Miles, 2010
Population per Square Mile
# Municipalities, 2018
With Population fewer Than 2,500
With Population of 2,500 to 4,999
With Population 5,000 to 9,999
With Population 10,000+
# Persons <18, 2018
% Population <18 Years Old
# Persons 18 to 64, 2018
% Population 18 to 64 Years Old
# Persons 65+, 2018
% Population 65+ Years Old
Median Age, 2018
# Persons White Only, 2018
% Population White Only
# Persons Black/African American Only, 2018
% Population Black/African American Only
# Persons Asian Only, 2018
% Population Asian Only
# Persons of Other Races and Two or More Races, 2018
% Population Other Races and Two or More Races
# Person of Hispanic/Latino Origin (All Races), 2018
% Population Hispanic/Latino Origin

In [9]:
voting_df = data[["Count of Democratic Voters", "Count of Republican Voters", "Count of No Affiliation Voters", "Voter Turnout, Nov. 2016"]]

general_population_df = data[["Population, 2019", "% Change, 2010-19",
                              "Natural Population Change (Births Minus Deaths), 2010 to 2019",
                              "Net International Migration",
                              "Net Domestic Migration",
                              "Total Net Migration"]]
municipalities_df = data[["# Municipalities, 2018", "With Population fewer Than 2,500", "With Population of 2,500 to 4,999",
                          "With Population 5,000 to 9,999", "With Population 10,000+", 
                         " # of Municipalities, 2018", "% With Municipal Comprehensive Plans", "% With Municipal Planning Commissions",
                          "% With Municipal and/or County Zoning Ordinances"]]

age_df = data[["# Persons <18, 2018","% Population <18 Years Old", "# Persons 18 to 64, 2018", "% Population 18 to 64 Years Old",
              "# Persons 65+, 2018",
              "% Population 65+ Years Old",
              "Median Age, 2018"]]


race_ethnicity_df = data[["# Persons White Only, 2018", "% Population White Only", "# Persons Black/African American Only, 2018",
                     "% Population Black/African American Only", "# Persons Asian Only, 2018", "% Population Asian Only",
                     "# Persons of Other Races and Two or More Races, 2018", "% Population Other Races and Two or More Races",
                     "# Person of Hispanic/Latino Origin (All Races), 2018", "% Population Hispanic/Latino Origin",
                     "% Native Born Residents, 2018", "% Foreign Born Residents, 2018"]]


households_and_familiesdf = data[["# Households, 2018", "# Families, 2018", "Total # Households, 2018",
                                 "% Married Couples With Own Children (<18)", "% Married Couples With No Children",
                                 "% Single Parents (Male/Female, No Spouse, Children <18)", "% Single Person Households",
                                 "% Other Types of Households", "% Households with One or More People <18 years old, 2000",
                                 "% Households with  One or More People <18 years old, 2018", 
                                  "Average # Persons in Household, 2018"]]

housing_by_unitdf = data[["# Housing Units, 2018", "% Change 2010 to 2018", "# Vacant Housing Units, 2018", "% Housing Units Vacant",
                         "% Vacant Units that are Seasonal, Recreational, or Occasional Use", 
                         "% Vacant Units for Rent/Sale ", "% Units Vacant for Other Reasons", "Total # Housing Units, 2018",
                         "% Single Family Home (Detached 1 Unit)", "% Duplex/Townhouse/Row Home (Attached 1 Unit)",
                         "% Small Apartment Type Building (Less than 9 Units)", "% Large Apartment Type Building (10 or More Units)",
                         "% Mobile Home & Other Types of Units", "# Occupied Housing Units, 2018", "% Homeownership Rate (Owner-Occupied Units)",
                         "% Renters (Renter-Occupied Units)", "New Housing Units Construction Permits, 2019"]]

housing_affordabilitydf = data[["Median Housing Value, 2018",
                                "Average Housing Value, 2018",
                                "Median Gross Monthly Rent, 2018",
                                "Average Gross Monthly Rent, 2018",
                                "Homeowners with Mortgage Whose Income/Housing Cost Ratios were Calculated, 2018",
                                "% Homeowners Paying Less than 30% of Income for Housing",
                                "% Homeowners Paying 30% to 49% of Income for Housing",
                                "% Homeowners Paying 50%+ of Income for Housing",
                                "# Renters For Whom Income/Rent Ratios were Calculated, 2018",
                                "% Renters Paying Less than 30% of Income for Housing",
                                "% Renters Paying 30% to 49% of Income for Housing",
                                "% Renters Paying 50%+ of Income for Housing"]]


household_incomedf = data[["# Households, 2018.1",
                            "% Households with Internet Access",
                            "% Households with No Internet Access",
                            "Median Household Income, 2018",
                            "# Households with Income, 2018",
                            "<$25,000 Household Income",
                            "$25,000 to $49,999 Household Income",
                            "$50,000 to $99,999 Household Income",
                            "$100,000 to $149,999 Household Income",
                            "$150,000+ Household Income",
                            "Average Household Income, 2018",
                            "# Households with Income, 2018.1",
                            "% Households with Income from Wages & Salaries",
                            "% Households with Income from Social Security ", 
                            "% Households with Income from Supplemental Security Income (SSI)",
                            "% Households with Income from Public Assistance",
                            "% Households with Income from Retirement "]] 



In [10]:
income_and_GDPdf = data[["Per Capita Personal Income, 2018",
                            "Total Personal Income ($1,000), 2018",
                            "% Income from Wages & Salaries",
                            "% Income from Dividends, Interest, & Rent",
                            "% Income from Transfer Payments",
                            "Gross Domestic Product (in $ Millions), 2018",
                            "Gross Domestic Product Per Capita, 2018",
                            "Total Poverty Rate, 2018",
                            "Poverty Rate for Children (<18 years old), 2018",
                            "Population Receiving Cash Assistance, June 2019 (Est.)"]]


public_programsdf = data[["Population Eligible for Medical Assistance (MA), June 2019",
                            "Population Participating in SNAP, June 2019",
                            "Total Number Enrolled in PACE/PACENET, 2018",
                            "% Population 65+ Enrolled in PACE/PACENET, 2018"]]


disabilitydf = data[["# Non-Institutionalized Persons 18 to 64 with Disability, 2018",
                        "% Population 18 to 64 with Disability",
                        "# Non-Institutionalized Persons 65+ with Disability, 2018",
                        "% Population 65+ with Disability"]]




In [11]:
educationdf = data[["# Persons, 25 Years Old & Older, 2018",
                    "No High School Diploma",
                    "High School Diploma or Equivalency",
                    "Some College, No Degree",
                    "Associate's Degree",
                    "Bachelor's Degree or Higher",
                    "Total # Students Enrolled in Grades K-12, 2018",
                    "% Enrolled in Public Schools",
                    "% Enrolled in Private Schools",
                    "# School Districts",
                    "# Students (Average Daily Membership), 2017-18",
                    "% Change, 2008-2018",
                    "Projected Enrollment, 2023-2024",
                    "Projected Enrollment, 2028-2029",
                    "Total School Revenues  ($1,000), 2017-18",
                    "% Revenues from Local Sources",
                    "% Revenues from State Sources",
                    "% Revenues from Federal & Other Sources",
                    "Total Expenditures Per Student, 2017-18",
                    "# to Receive Free & Reduced Lunch, 2018",
                    "Graduation Rate (4-year Cohort), 2017-18",
                    "# Degree Granting Institutions (Colleges & Universities), 2018",
                    "# Non-Degree Granting Institutions (Trade & Technical schools), 2018"]]

employmentdf = data[["Total # Employed Persons, 2018",
                        "% Manufacturing",
                        "% Wholesale & Retail Trade",
                        "% Mining, Construction, Utilities, & Transportation & Warehousing",
                        "% Information, Finance & Insurance, & Real Est. & Rental & Leasing",
                        "% Education Serv. & Professional, Scientific, & Technical Services",
                        "% Health Care & Social Assistance",
                        "% Accommodation & Food Services",
                        "% Other Sectors including Public Administration",
                        "# Employed Persons, 2018",
                        "% Management, Professional, and Related",
                        "% Service",
                        "% Sales and Office",
                        "% Farming, Fishing, and Forestry",
                        "% Construction, Extraction, Maintenance, and Repair",
                        "% Production, Transportation, and Material Moving",
                        "Average Unemployment Rate, 2019",
                        "Average # Business Establishments, 2nd Qt. 2019",
                        "Average Employment, 2nd Qt. 2019",
                        "Average # Employed Persons in Workforce, 2018",
                        "% of Employed Persons <25 Years Old",
                        "% Employed Persons 25 to 54 Years Old",
                        "% Employed Persons 55+ Years Old",
                        "# Employed Persons 16 Years Old & Older, 2018",
                        "% Who Worked in County of Residence",
                        "% Who Worked Outside County of Residence, but Within PA",
                        "% Who Worked Outside Pennsylvania",
                        "Avg. Time to Work (Minutes), 2018",
                        "Total # Filings for Chapters 7, 11, 12, and 13, 2019"]]




In [12]:
birthdf = data[[" Live Births Per 1,000 Residents, 2018",
                " Deaths Per 1,000 Residents (Preliminary), 2018",
                "# Reported Pregnancies, 2015-17",
                "% Resulting in Live Births",
                "% Ending in Fetal Deaths",
                "% Ending in Induced Abortions",
                "% Reported Pregnancies of Women <18 Years Old",
                "% Born With Low Birth Weight (Under 2,500 Grams), 2016-2018",
                "% Born to Unmarried Mothers, 2016-2018",
                "% Born to Mothers Who Received Medicaid, 2016-2018",
                "% Born to Mothers Who Did Not Receive Prenatal Care in 1st Trimester, 2016-2018"]]

deathdf = data[["Resident Deaths, 2015-2017",
                "Death Rate Per 1,000 Residents, All Causes",
                "Heart Disease Death Rate per 1,000 Residents",
                "Cancer Death Rate per 1,000 Residents",
                "Drug Overdose Death Rate per 1,000 Residents",
                "Suicide Death Rate per 1,000 Residents"]]

medical_caredf = data[[" General Acute Care Hospitals, 2018",
                        "Hospital Beds Set Up & Staffed, 2018",
                        " Beds Set Up & Staffed per 1,000 Residents",
                        "# Nursing Homes, 2017-18",
                        "Total # Licensed/Approved Nursing Home Beds, 2017-18",
                        "Total # Licensed/Approved Nursing Home Beds per 1,000 Residents, 2017-18",
                        "# Primary Care Physicians (MD/DO), 2017",
                        "# Primary Care Physicians per 100,000 Residents, 2017",
                        "# Nurse Practitioners, 2017",
                        "# Nurse Practitioners per 100,000 Residents, 2017",
                        "# Dentists, 2017",
                        "# Dentists per 100,000 Residents, 2017",
                        "%  Without Health Insurance <65 Years Old, 2017"]]

policing_df = data[["Serious Crimes per 100,000 Residents, 2017",
                    "Full-Time Police Officers per 1,000 Residents, 2017"]]

highway_and_drivingdf = data[["Total Miles of Highway, 2018",
                            "% PennDOT and Other State & Federal Highways",
                            "% Local (Municipal) Highways",
                            "# Licensed Drivers, January, 2019",
                            "% Change in Licensed Drivers, 2009-2019",
                            " In-State Registered Vehicles, 2018",
                            " In-State Registered Vehicles Per 1,000 Residents",
                            "Change in In-State Registered Vehicles, 2008-2018"]]

farmdf = data[["# Farms, 2017",
                "Acres in Farmland, 2017",
                "% Land In Farms, 2017",
                "Average Size of Farms (acres), 2017",
                "Total Market Value of Ag. Products Sold ($1,000), 2017",
                "Average Market Value of Products Sold Per Farm, 2017",
                "Average Market Value of Products Sold Per Acre, 2017",
                "Farm Sales Under $10,000, 2017",
                "Farm Sales $10,000-$49,999, 2017",
                "Farm Sales $50,000+, 2017",
                "Preserved Farms, April 2017",
                "Acres of Preserved Farmland, April 2017"]]

randomdf = data[["Population",
                    "Birth",
                    "Cancer",
                    "Death Rate",
                    "GDP"]]




In [13]:
total_dfs = [{'name': "Voter Data" , 'data': voting_df},
            {'name': "General Population Data" , 'data': general_population_df},
            {'name': "Municipal Data" , 'data': municipalities_df},
            {'name': "Age Data" , 'data': age_df},
            {'name': "Race/Ethnicity Data" , 'data': race_ethnicity_df},
            {'name': "Family Unit Data" , 'data': households_and_familiesdf},
            {'name': "Housing Unit Data" , 'data': housing_by_unitdf},
            {'name': "Housing Affordability Data" , 'data': housing_affordabilitydf},
            {'name': "Household Income Data" , 'data': household_incomedf},
            {'name': "Economic Data" , 'data': income_and_GDPdf},
            {'name': "Public Program Data" , 'data': public_programsdf},
            {'name': "Disability Data" , 'data': disabilitydf},
            {'name': "Education Data" , 'data': educationdf},
            {'name': "Employment Data" , 'data': employmentdf},
            {'name': "Birth Data" , 'data': birthdf},
            {'name': "Death Data" , 'data': deathdf},
            {'name': "Medical Care Data" , 'data': medical_caredf},
            {'name': "Policing Data" , 'data': policing_df},
            {'name': "Highway and Driving Data" , 'data': highway_and_drivingdf},
            {'name': "Farm Data" , 'data': farmdf},
            {'name': "Randomly Associated Test Data" , 'data': randomdf}]



In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
# Create a StandardScater model and fit it to the training data

def runRFM(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    rf = RandomForestClassifier(n_estimators=200)
    rf = rf.fit(X_train_scaled, y_train)
    rf_feature_importances = sorted(zip(rf.feature_importances_, feature_names), reverse=True)
    accuracy = rf.score(X_test_scaled, y_test)
    output = {'Importances': rf_feature_importances, 'Accuracy': accuracy}
    
    return output

In [41]:
def RFMiterator1(alldatasets, total_iterations, target_col):
        
    for iteration in range(total_iterations):
        
        land_accuracies = [] 
        dataset_names = []
    
        for i in range(len(alldatasets)):

            rfm_output1 = runRFM(alldatasets[i]['data'], target_col)

#             print("\nBegin Calculation for " + alldatasets[i]['name'])
#             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#             print('Overall Importances: ')
#             for importance in rfm_output1['Importances']:
#                 print(importance)
            land_accuracies.append(rfm_output1['Accuracy'])
            dataset_names.append(alldatasets[i]['name'])
#             print('End Calculation\n')
        
        if(iteration == 0):
        
            accuracies = pd.DataFrame({"Dataset Names": dataset_names, "Land Accuracies 0": land_accuracies})
            
        accuracies['Land Accuracies ' + str(iteration)] = land_accuracies

    
    print('End Calculation\n') 
    
    return accuracies
    

In [65]:
def RFMiterator2(alldatasets, total_iterations, target_col):
        
    for iteration in range(total_iterations):
        
        ghg_accuracies = [] 
        dataset_names = []
    
        for i in range(len(alldatasets)):

            rfm_output2 = runRFM(alldatasets[i]['data'], target_col)

#             print("\nBegin Calculation for " + alldatasets[i]['name'])
#             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#             print('Overall Importances: ')
#             for importance in rfm_output1['Importances']:
#                 print(importance)
            ghg_accuracies.append(rfm_output2['Accuracy'])
            dataset_names.append(alldatasets[i]['name'])
#             print('End Calculation\n')
        
        if(iteration == 0):
        
            accuracies = pd.DataFrame({"Dataset Names": dataset_names, "GHG Accuracies 0": ghg_accuracies})
            
        accuracies['GHG Accuracies ' + str(iteration)] = ghg_accuracies

    
    print('End Calculation\n') 
    
    return accuracies
    

In [42]:
statisticalset1 = RFMiterator1(total_dfs, 50, target1)

End Calculation



In [43]:
statisticalset1

,Dataset Names,Land Accuracies 0,Land Accuracies 1,Land Accuracies 2,Land Accuracies 3,Land Accuracies 4,Land Accuracies 5,Land Accuracies 6,Land Accuracies 7,Land Accuracies 8,...,Land Accuracies 40,Land Accuracies 41,Land Accuracies 42,Land Accuracies 43,Land Accuracies 44,Land Accuracies 45,Land Accuracies 46,Land Accuracies 47,Land Accuracies 48,Land Accuracies 49
0,Voter Data,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
1,General Population Data,0.6250,0.6875,0.6875,0.6250,0.6875,0.6875,0.5625,0.5625,0.6250,...,0.6250,0.6250,0.6250,0.6250,0.6250,0.6250,0.5625,0.6250,0.6250,0.6250
2,Municipal Data,0.6875,0.6875,0.6875,0.6875,0.6250,0.6875,0.6875,0.6875,0.6875,...,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875
3,Age Data,0.6875,0.6875,0.6875,0.6875,0.7500,0.6875,0.6875,0.6875,0.6875,...,0.6875,0.6875,0.6875,0.6250,0.6875,0.6250,0.6875,0.6875,0.6875,0.7500
4,Race/Ethnicity Data,0.7500,0.7500,0.7500,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,...,0.6875,0.6875,0.6250,0.7500,0.7500,0.6875,0.7500,0.7500,0.6875,0.7500
5,Family Unit Data,0.6875,0.6875,0.6875,0.6250,0.6875,0.6875,0.6250,0.6250,0.6875,...,0.6875,0.6250,0.6875,0.6250,0.6875,0.6875,0.6250,0.6250,0.6250,0.6875
6,Housing Unit Data,0.6250,0.5625,0.6250,0.7500,0.6875,0.6875,0.6875,0.6250,0.6250,...,0.6875,0.6875,0.6875,0.7500,0.5625,0.6875,0.6875,0.7500,0.6250,0.6250
7,Housing Affordability Data,0.6875,0.7500,0.7500,0.7500,0.6875,0.6250,0.6875,0.7500,0.7500,...,0.7500,0.6875,0.6875,0.6875,0.7500,0.7500,0.7500,0.6875,0.6875,0.6875
8,Household Income Data,0.6250,0.6250,0.6250,0.6250,0.5625,0.6250,0.6250,0.6250,0.5625,...,0.6875,0.5625,0.6250,0.5625,0.6250,0.6875,0.6250,0.5625,0.5625,0.6250
9,Economic Data,0.6875,0.6875,0.6875,0.6875,0.6875,0.6250,0.6250,0.6875,0.6875,...,0.6875,0.6250,0.6875,0.6250,0.6250,0.6250,0.7500,0.6875,0.6875,0.6875


In [62]:
statisticalset1['Average'] = statisticalset1.mean('columns')
statisticalset1['Max'] = statisticalset1.max('columns')
statisticalset1['Min'] = statisticalset1.min('columns')
statisticalset1['Standard Deviation'] = statisticalset1.std('columns')
statisticalset1['Variance'] = statisticalset1.var('columns')

statisticalset1[['Dataset Names','Average', 'Standard Deviation', 'Variance', 'Max', 'Min']]

,Dataset Names,Average,Standard Deviation,Variance,Max,Min
0,Voter Data,0.50000,0.000000,0.004630,0.5000,0.5000
1,General Population Data,0.63250,0.040008,0.008065,0.6875,0.5625
2,Municipal Data,0.68375,0.016655,0.008488,0.6875,0.6250
3,Age Data,0.69625,0.036831,0.009375,0.7500,0.6250
4,Race/Ethnicity Data,0.71125,0.038817,0.009829,0.7500,0.6250
5,Family Unit Data,0.66375,0.030407,0.008329,0.6875,0.6250
6,Housing Unit Data,0.66000,0.052597,0.009543,0.7500,0.5625
7,Housing Affordability Data,0.72250,0.035655,0.009950,0.7500,0.6250
8,Household Income Data,0.60125,0.040706,0.007463,0.6875,0.5625
9,Economic Data,0.66875,0.033200,0.008578,0.7500,0.6250


In [66]:
statisticalset2 = RFMiterator2(total_dfs, 50, target2)

End Calculation



In [67]:
statisticalset2

,Dataset Names,GHG Accuracies 0,GHG Accuracies 1,GHG Accuracies 2,GHG Accuracies 3,GHG Accuracies 4,GHG Accuracies 5,GHG Accuracies 6,GHG Accuracies 7,GHG Accuracies 8,...,GHG Accuracies 40,GHG Accuracies 41,GHG Accuracies 42,GHG Accuracies 43,GHG Accuracies 44,GHG Accuracies 45,GHG Accuracies 46,GHG Accuracies 47,GHG Accuracies 48,GHG Accuracies 49
0,Voter Data,0.5000,0.4375,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.4375,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
1,General Population Data,0.4375,0.5000,0.3750,0.3750,0.6250,0.4375,0.5625,0.5000,0.5625,...,0.5000,0.5000,0.5625,0.3750,0.5000,0.5625,0.3750,0.3750,0.4375,0.5625
2,Municipal Data,0.5000,0.4375,0.5000,0.5000,0.5625,0.5625,0.5000,0.4375,0.4375,...,0.5000,0.5000,0.5000,0.4375,0.5000,0.4375,0.5625,0.5625,0.5000,0.5000
3,Age Data,0.6250,0.6250,0.5625,0.6250,0.5625,0.5625,0.6250,0.5625,0.6250,...,0.6875,0.5000,0.6250,0.6875,0.6250,0.5000,0.5000,0.6875,0.6875,0.6250
4,Race/Ethnicity Data,0.4375,0.5000,0.4375,0.5000,0.5000,0.5000,0.5000,0.4375,0.4375,...,0.4375,0.5000,0.4375,0.5625,0.5000,0.4375,0.4375,0.5000,0.4375,0.5000
5,Family Unit Data,0.6875,0.6875,0.6875,0.6875,0.6875,0.6250,0.6250,0.6250,0.5625,...,0.6875,0.5625,0.6250,0.6250,0.6250,0.6875,0.6875,0.6250,0.6875,0.6250
6,Housing Unit Data,0.5000,0.3750,0.4375,0.4375,0.4375,0.4375,0.3750,0.4375,0.5000,...,0.4375,0.3750,0.3750,0.3750,0.5625,0.3750,0.3750,0.4375,0.5000,0.4375
7,Housing Affordability Data,0.5625,0.5625,0.5625,0.5000,0.6250,0.5625,0.6250,0.5625,0.6250,...,0.6250,0.5000,0.5000,0.5625,0.5625,0.5625,0.6250,0.4375,0.5000,0.6250
8,Household Income Data,0.3750,0.5000,0.5000,0.4375,0.3750,0.4375,0.4375,0.5000,0.4375,...,0.3750,0.4375,0.4375,0.5000,0.4375,0.4375,0.3750,0.3750,0.4375,0.5000
9,Economic Data,0.3750,0.3750,0.4375,0.3750,0.5000,0.5625,0.4375,0.4375,0.3125,...,0.3750,0.4375,0.4375,0.3750,0.4375,0.4375,0.3750,0.4375,0.3750,0.4375


In [68]:
statisticalset2['Average'] = statisticalset2.mean('columns')
statisticalset2['Max'] = statisticalset2.max('columns')
statisticalset2['Min'] = statisticalset2.min('columns')
statisticalset2['Standard Deviation'] = statisticalset1.std('columns')
statisticalset2['Variance'] = statisticalset2.var('columns')

statisticalset2[['Dataset Names','Average', 'Standard Deviation', 'Variance', 'Max', 'Min']]

,Dataset Names,Average,Standard Deviation,Variance,Max,Min
0,Voter Data,0.49250,0.094023,0.003778,0.5625,0.4375
1,General Population Data,0.48500,0.121458,0.007115,0.6250,0.3750
2,Municipal Data,0.49875,0.127655,0.004539,0.5625,0.4375
3,Age Data,0.60125,0.132173,0.007780,0.7500,0.5000
4,Race/Ethnicity Data,0.46750,0.135114,0.004035,0.5625,0.3750
5,Family Unit Data,0.64500,0.125305,0.006688,0.6875,0.5625
6,Housing Unit Data,0.42500,0.129585,0.004722,0.5625,0.3125
7,Housing Affordability Data,0.56750,0.136519,0.006804,0.6875,0.4375
8,Household Income Data,0.43000,0.116327,0.003732,0.5000,0.3125
9,Economic Data,0.42625,0.126806,0.005899,0.5625,0.3125


In [45]:
#https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
# land_accuracies = [] 
# ghg_rate_accuracies = [] 
# dataset_names = []


# for i in range(len(total_dfs)):

#     rfm_output1 = runRFM(total_dfs[i]['data'], target1)

#     print("\nBegin Calculation for " + total_dfs[i]['name'])
#     print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#     print('Overall Importances: ')
#     for importance in rfm_output1['Importances']:
#         print(importance)
#     land_accuracies.append(rfm_output1['Accuracy'])
#     dataset_names.append(total_dfs[i]['name'])
#     print('End Calculation\n')

    

In [44]:
#https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
# for i in range(len(total_dfs)):
    
#     rfm_output2 = runRFM(total_dfs[i]['data'], target2)
    
#     print("\nBegin Calculation for " + total_dfs[i]['name'])
#     print('Overall Model Accuracy: ', rfm_output2['Accuracy'])
#     print('Overall Importances: ')
#     for importance in rfm_output2['Importances']:
#         print(importance)
#     ghg_rate_accuracies.append(rfm_output2['Accuracy'])
#     print('End Calculation\n')

In [70]:
avg_accuracy_df = pd.DataFrame({'Dataset Names': statisticalset1['Dataset Names'], 'Land Classifier Avg. Accuracy': statisticalset1['Average'],
                               'CO2 Rate Avg. Accuracy': statisticalset2['Average']})

avg_accuracy_df

,Dataset Names,Land Classifier Avg. Accuracy,CO2 Rate Avg. Accuracy
0,Voter Data,0.50000,0.49250
1,General Population Data,0.63250,0.48500
2,Municipal Data,0.68375,0.49875
3,Age Data,0.69625,0.60125
4,Race/Ethnicity Data,0.71125,0.46750
5,Family Unit Data,0.66375,0.64500
6,Housing Unit Data,0.66000,0.42500
7,Housing Affordability Data,0.72250,0.56750
8,Household Income Data,0.60125,0.43000
9,Economic Data,0.66875,0.42625
